In [18]:
from datasets import load_dataset
import pandas as pd

# Загружаем данные (берем чуть больше, чтобы после удаления дублей осталось ровно 50)
ru_data = load_dataset("sberquad", split="train", streaming=True)

# Используем set для хранения только уникальных текстов
unique_texts = set()

for entry in ru_data:
    text = entry["context"].strip()
    unique_texts.add(text)


# Превращаем в список словарей для DataFrame
raw_texts = [{"text": t} for t in unique_texts]

# Создаем DataFrame и перемешиваем
df = pd.DataFrame(raw_texts).sample(frac=1, random_state=42)
df.to_json("raw_texts.jsonl", orient="records", lines=True, force_ascii=False)

print(f"Готово! Файл создан. Всего уникальных записей: {len(df)}")

Готово! Файл создан. Всего уникальных записей: 9078


In [19]:
raw_texts

[{'text': 'Псевдоним Ричард Бахман был разоблачен продавцом книжного магазина из Вашингтона, Стивом Брауном, который заметил сходство между старыми работами Кинга и новыми работами Бахмана, он обнаружил фамилию Кинга на одном из романов Бахмана в Библиотеке Конгресса[35]. Это привело к пресс-релизу, предвещающему скорую смерть Бахмана, якобы от рака . У этого псевдонима есть вымышленная биография. Книги Бахмана, якобы покойного (скончавшегося от рака псевдонима [36]), были изданы его также вымышленной вдовой, Клаудией Иннес Бахман. Интересно, что Клаудия упоминается в цикле Тёмная Башня как автор книги Чарли Чу-Чу в ключевом мире (в других мирах автор — вымышленная Берил Эванз). В книге её имя пишется иначе (Клаудия-и-Иннес Бахман — 19 букв). Можно сказать, что и Клаудия Бахман — также псевдоним Кинга. Кинг использовал и другие псевдонимы, такие как Джон Свитен для романа Пятая четверть . Кинг посвятил этой истории свою книгу 1989 года Тёмная половина о том, как псевдоним обрел плоть и

In [5]:
from google.colab import userdata
API_KEY= userdata.get('API_KEY_SGR')

In [9]:
import json
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

In [22]:
import sqlite3
import json
from openai import OpenAI

# Настройка БД
conn = sqlite3.connect('rag_eval.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS statements_data
                  (id INTEGER PRIMARY KEY, source_text TEXT UNIQUE, statements_json TEXT)''')
conn.commit()

def get_statements(source_text):
    system_instruction = (
        "Ты — эксперт по извлечению фактов. Твоя цель — разбить входной текст на атомарные, "
        "независимые утверждения. Важно: заменяй все местоимения (он, она, оно, они и т. д.) "
        "на конкретные субъекты, к которым они относятся. Каждое утверждение должно быть самодостаточным. "
        "Возвращай результат строго в виде JSON-объекта с ключом 'statements', содержащим список строк. "
        "Язык ответа должен соответствовать языку входного текста."
    )

    completion = client.chat.completions.create(
        model="z-ai/glm-4.5-air:free",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": source_text}
        ],
        response_format={"type": "json_object"}
    )

    # Возвращаем только содержимое сообщения (JSON-строку)
    return completion.choices[0].message.content

# Цикл обработки
for item in raw_texts[:50]:
    text = item['text']

    # ШАГ 1: Проверяем наличие текста в БД перед вызовом функции
    cursor.execute("SELECT 1 FROM statements_data WHERE source_text = ?", (text,))
    exists = cursor.fetchone()

    if exists:
        print(f"Пропуск: текст уже обработан и есть в базе.")
        continue # Переходим к следующему итератору, get_statements() не вызывается

    # ШАГ 2: Если текста нет, вызываем LLM и сохраняем
    try:
        print(f"Обработка нового текста...")
        result_json = get_statements(text)

        # Валидация JSON
        json.loads(result_json)

        cursor.execute("INSERT INTO statements_data (source_text, statements_json) VALUES (?, ?)",
                       (text, result_json))
        conn.commit()
        print("Данные успешно сохранены.")
    except Exception as e:
        print(f"Ошибка при обработке: {e}")

# Закрываем соединение в самом конце
conn.close()
print("Сбор данных завершен.")

Пропуск: текст уже обработан и есть в базе.
Пропуск: текст уже обработан и есть в базе.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...
Данные успешно сохранены.
Обработка нового текста...


KeyboardInterrupt: 

In [16]:
raw_texts[:10]

[{'text': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.'},
 {'text': 'В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в резул